In [1]:

import torch
import pandas as pd
import numpy as np
import boto3
import numpy as np
import pandas as pd
from utils import get_args_parser
from remasker_impute import ReMasker
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt

s3 = boto3.client('s3')
bucket_name = 'sagemaker-studio-905418013525-nvxe84zgs6'
file_path = 'Labrado/alllabs1000adm/Labitemlist.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
Labitemlist = pd.read_csv(obj['Body'], header=None,names=['labitem', 'cnt'], dtype=int)

file_path = 'Labrado/alllabs1000adm/unique_adm_year_day.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
unique_adm_year_day = pd.read_csv(obj['Body'], header=None,names=['hadm_id', 'chartyear', 'day','rowinx'], dtype=int)

unique_hadm_by_year = unique_adm_year_day.groupby('chartyear')['hadm_id'].nunique()
cumulative_sum = unique_hadm_by_year.cumsum()
total_sum = cumulative_sum.iloc[-1]
split_index = cumulative_sum[cumulative_sum >= 0.8 * total_sum].index[0]
print("split year is",split_index)
Row_Train = unique_adm_year_day['chartyear']<=split_index
Train_inx = Row_Train[Row_Train].index
Row_Test = unique_adm_year_day['chartyear']>split_index
Test_inx = Row_Test[Row_Test].index

file_path = 'Labrado/alllabs1000adm/npval.csv'
obj = s3.get_object(Bucket=bucket_name, Key=file_path)
npval = pd.read_csv(obj['Body'], header=None, dtype=float)
npval_train = npval.iloc[Train_inx[:10000],:10]
npval_test = npval.iloc[Test_inx[:10000],:10]




split year is 2178


In [2]:
npval_test.head()

,0,1,2,3,4,5,6,7,8,9
11,4.3,137.0,102.0,38.4,1.0,20.0,19.0,19.0,77.0,154.0
12,3.2,141.0,103.0,38.2,1.1,31.0,10.0,22.0,156.0,146.0
13,3.4,145.0,103.0,39.0,1.0,30.0,15.0,18.0,99.0,159.0
55,4.0,138.0,103.0,37.8,0.8,28.0,11.0,13.0,108.0,172.0
56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
npval_test = npval_test.rename(columns={
    0: 'Crt', 
    1: 'Plt',
    2:'Hgb',
    3:'Wbc',
    4:'Ure',
    5:'50983',
    6:'Bic',
    7:'50971',
    8:'50902',
    9:'51221',
    })
npval_train = npval_train.rename(columns={
    0: 'Crt', 
    1: 'Plt',
    2:'Hgb',
    3:'Wbc',
    4:'Ure',
    5:'50983',
    6:'Bic',
    7:'50971',
    8:'50902',
    9:'51221',
    })

In [9]:
npval_test.head()

,Crt,Plt,Hgb,Wbc,Ure,50983,Bic,50971,50902,51221
11,4.3,137.0,102.0,38.4,1.0,20.0,19.0,19.0,77.0,154.0
12,3.2,141.0,103.0,38.2,1.1,31.0,10.0,22.0,156.0,146.0
13,3.4,145.0,103.0,39.0,1.0,30.0,15.0,18.0,99.0,159.0
55,4.0,138.0,103.0,37.8,0.8,28.0,11.0,13.0,108.0,172.0
56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
npval_train.to_csv("sample_train.csv")
npval_test.to_csv("sample_test.csv")